Preprocesing Text menggunakan NLTK

Nicholaus Verdhy || A11.2020.12447

# Penjelasan Singkat mengenai NLTK

NLTK adalah kumpulan alat dan program yang digunakan untuk memproses dan menganalisis bahasa alami (NLP) dalam bahasa Inggris. Ini ditulis dalam bahasa pemrograman Python dan digunakan untuk tugas-tugas NLP yang melibatkan analisis teks dan statistik[1]

# Project 1 : Preprocessing dengan NLTK

## Import Library

In [8]:
import re
import string
import time
import pandas as pd
from copy import deepcopy


#Untuk Preprocessing Data nya
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

#import nltk
import nltk
from nltk.tokenize import word_tokenize

#ignore warning
import warnings
warnings.filterwarnings('ignore')

Penjelasan untuk setiap library
- `import re` digunakan untuk bekerja dengan ekspresi reguler (regular expressions), yang memungkinkan Anda untuk melakukan pencarian dan manipulasi teks berdasarkan pola-pola tertentu.digunakan untuk pencarian dan manipulasi teks berdasarkan pola pola tertentu
- `import string` digunakan dalam manipulasi teks, seperti menghapus tanda baca atau mengoperasikan karakter-karakter tertentu 
- `import time` digunakan untuk bekerja dengan waktu dan tanggal dalam Python.
- `import pandas` digunakan untuk memuat dataset yang ada
- `from copy import deepcopy`  ini gunanya, ketika kita melakukan duplikasi dari objek, maka hasil duplikasinya adalah objek baru dan tidak terpengaruhi oleh objek lama
- `from ekphrasis.classes.preprocessor import TextPreProcessor` untuk melakukan berbagai tugas pemrosesan teks, terutama pada teks yang berasal dari media sosial.
- `from ekphrasis.classes.tokenizer import SocialTokenizer`adalah tokenizer yang digunakan untuk membagi teks menjadi kata-kata
- `from ekphrasis.dicts.emoticons import emoticons` adalah kamus yang mungkin digunakan untuk mengganti emotikon dengan kata-kata yang sesuai.
- `import nltk` adalah pernyataan impor untuk mengimpor modul NLTK (Natural Language Toolkit), yang merupakan pustaka yang digunakan dalam pemrosesan bahasa alami (NLP)
- `from nltk.tokenize import word_tokenize` adalah salah satu alat dalam NLTK yang digunakan untuk membagi teks menjadi kata-kata atau token. 
- `import warning` mengabaikan warning

## Loading Dataset

In [4]:
df = pd.read_csv("Dataset_Sentimen_Emosi.csv")

In [5]:
df.head()

,Tweet,Sentimen,Emosi
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,1
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,-1
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,1
3,Covid belum nyampe prigen mbak hmm hoax,0.0,-2
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,-2


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     904 non-null    object 
 1   Sentimen  903 non-null    float64
 2   Emosi     904 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 21.3+ KB


Deskripsi Kolom:

`Tweet` : Isi hasil tweet orang di twiter

`Sentimen`  : mengindikasikan sentimen atau perasaan yang terkait dengan setiap tweet. Sentimen dapat berupa positif, negatif, netral, atau mungkin memiliki label lebih rinci seperti senang, marah, sedih, dan lain-lain.

`Emosi`      : sama seperti sentimen

## Preprocessing Data

Sumber modul : https://github.com/cbaziotis/ekphrasis 

Modulnya ekphrasis

In [9]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


# Referensi

[1]	E. J. Rifano, A. C. Fauzan, A. Makhi, E. Nadya, Z. Nasikin, and F. N. Putra, “Text Summarization Menggunakan Library Natural Language Toolkit (NLTK) Berbasis Pemrograman Python,” Ilk. J. Comput. Sci. Appl. Inform., vol. 2, no. 1, Art. no. 1, Apr. 2020, doi: 10.28926/ilkomnika.v2i1.32.

